In [1]:
import os
import time
import pandas as pd
import numpy as np
from h2o.automl import H2OAutoML
import h2o

In [2]:
RANDOM_STATE = 42
N_SYNTHETIC_SAMPLES_TO_GENERATE = 250000 
MODEL_CONFIDENCE_THRESHOLD = 0.75
BATCH_SIZE = 5000  
MAX_GENERATION_ATTEMPTS = 1000000 

IMBALANCED_DATA_PATH = '../data/01_raw/original_imbalanced.csv'
PROCESSED_DATA_DIR = '../data/02_processed/'
MODEL_DRIVEN_SYNTHETIC_PATH = os.path.join(PROCESSED_DATA_DIR, 'model_driven_synthetic.csv')
MODELS_DIR = '../models/'
os.makedirs(MODELS_DIR, exist_ok=True)


In [ ]:
h2o.init(nthreads=8, max_mem_size="12g") 
df_imbalanced = pd.read_csv(IMBALANCED_DATA_PATH)
hf_imbalanced = h2o.H2OFrame(df_imbalanced)
response = "target"
predictors = [c for c in hf_imbalanced.columns if c != response]
hf_imbalanced[response] = hf_imbalanced[response].asfactor()
train_i, valid_i, test_i = hf_imbalanced.split_frame(ratios=[0.7, 0.15], seed=RANDOM_STATE)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM Temurin-17.0.16+8 (build 17.0.16+8, mixed mode, sharing)
  Starting server from C:\Project\Synthetic Intelligence\.venv\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\bharg\AppData\Local\Temp\tmpxs38570q
  JVM stdout: C:\Users\bharg\AppData\Local\Temp\tmpxs38570q\h2o_bharg_started_from_python.out
  JVM stderr: C:\Users\bharg\AppData\Local\Temp\tmpxs38570q\h2o_bharg_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/London
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,5 months and 6 days
H2O_cluster_name:,H2O_from_python_bharg_3u0fnh
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
# Load or train model
leader_model_path = os.path.join(MODELS_DIR, "imbalanced_leader.model")
if os.path.exists(leader_model_path):
    print("Loading saved imbalanced model...")
    imbalanced_leader_model = h2o.load_model(leader_model_path)
else:
    print("Training and saving imbalanced model...")
    aml_imb = H2OAutoML(
        max_runtime_secs=3600, max_models=20, seed=RANDOM_STATE, 
        project_name=f"imbalanced_for_generation", sort_metric="AUCPR"
    )
    aml_imb.train(x=predictors, y=response, training_frame=train_i, validation_frame=valid_i)
    imbalanced_leader_model = aml_imb.leader
    h2o.save_model(model=imbalanced_leader_model, path=MODELS_DIR, 
                   filename="imbalanced_leader.model", force=True)

print("Imbalanced Leader Model for Generation:")
print(imbalanced_leader_model.model_id)


Loading saved imbalanced model...
Imbalanced Leader Model for Generation:
StackedEnsemble_AllModels_1_AutoML_1_20250902_130655


In [ ]:
print(f"Starting optimized model driven synthetic data generation for {N_SYNTHETIC_SAMPLES_TO_GENERATE} samples...")
df_train = train_i.as_data_frame()
df_minority = df_train[df_train['target'] == 1].drop('target', axis=1)
df_majority = df_train[df_train['target'] == 0]

numerical_cols = df_minority.select_dtypes(include=np.number).columns.tolist()
categorical_cols = df_minority.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"Minority samples available: {len(df_minority)}")
print(f"Numerical columns: {len(numerical_cols)}, Categorical columns: {len(categorical_cols)}")

Starting optimized model driven synthetic data generation for 250000 samples...


c:\Project\Synthetic Intelligence\.venv\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Minority samples available: 3966
Numerical columns: 30, Categorical columns: 10


In [ ]:
# Pre compute minority sample arrays for faster access
minority_numerical = df_minority[numerical_cols].values if numerical_cols else np.array([])
minority_categorical = df_minority[categorical_cols].values if categorical_cols else np.array([])
minority_indices = df_minority.index.values

In [ ]:
def generate_batch_samples(batch_size, minority_data_num, minority_data_cat, 
                          numerical_cols, categorical_cols, minority_indices):
    """Generate a batch of synthetic samples efficiently"""
    batch_samples = []
    
    # Generate all random choices at once
    if len(minority_indices) >= 2:
        # Generate pairs of indices
        p1_indices = np.random.choice(len(minority_indices), batch_size, replace=True)
        p2_indices = np.random.choice(len(minority_indices), batch_size, replace=True)
        
        mask = p1_indices == p2_indices
        while mask.any():
            p2_indices[mask] = np.random.choice(len(minority_indices), mask.sum(), replace=True)
            mask = p1_indices == p2_indices
        
        interpolation_ratios = np.random.rand(batch_size)
        categorical_choices = np.random.rand(batch_size) > 0.5
        
        for i in range(batch_size):
            new_sample = {}
            
            if len(numerical_cols) > 0:
                p1_num = minority_data_num[p1_indices[i]]
                p2_num = minority_data_num[p2_indices[i]]
                interpolated = p1_num * interpolation_ratios[i] + p2_num * (1 - interpolation_ratios[i])
                new_sample.update(dict(zip(numerical_cols, interpolated)))
            
            if len(categorical_cols) > 0:
                if categorical_choices[i]:
                    cat_values = minority_data_cat[p1_indices[i]]
                else:
                    cat_values = minority_data_cat[p2_indices[i]]
                new_sample.update(dict(zip(categorical_cols, cat_values)))
            
            batch_samples.append(new_sample)
    
    return batch_samples

synthetic_samples = []
total_attempts = 0

In [ ]:
print("Starting batch generation...")
while len(synthetic_samples) < N_SYNTHETIC_SAMPLES_TO_GENERATE and total_attempts < MAX_GENERATION_ATTEMPTS:
    remaining_samples = N_SYNTHETIC_SAMPLES_TO_GENERATE - len(synthetic_samples)
    current_batch_size = min(BATCH_SIZE, remaining_samples * 3)  
    
    
    batch_samples = generate_batch_samples(
        current_batch_size, minority_numerical, minority_categorical,
        numerical_cols, categorical_cols, minority_indices
    )
    
    if not batch_samples:
        break
        
    batch_df = pd.DataFrame(batch_samples)
    
    batch_df = batch_df.reindex(columns=predictors, fill_value=0)
    
    try:
        batch_hf = h2o.H2OFrame(batch_df)
        
        batch_predictions = imbalanced_leader_model.predict(batch_hf)
        batch_predictions_df = batch_predictions.as_data_frame()
        
        confident_mask = batch_predictions_df['p1'] >= MODEL_CONFIDENCE_THRESHOLD
        confident_samples = batch_df[confident_mask]
        
        # Add target column and append to results
        if len(confident_samples) > 0:
            confident_samples = confident_samples.copy()
            confident_samples['target'] = 1
            synthetic_samples.extend(confident_samples.to_dict('records'))
        
        total_attempts += current_batch_size
        
        batch_hf.remove()
        batch_predictions.remove()
        
        if total_attempts % (BATCH_SIZE * 2) == 0:
            acceptance_rate = len(synthetic_samples) / total_attempts * 100
            print(f"Attempts: {total_attempts}, Generated: {len(synthetic_samples)}/{N_SYNTHETIC_SAMPLES_TO_GENERATE} "
                  f"(Acceptance rate: {acceptance_rate:.2f}%)")
            
    except Exception as e:
        print(f"Error in batch processing: {e}")
        continue

if len(synthetic_samples) > N_SYNTHETIC_SAMPLES_TO_GENERATE:
    synthetic_samples = synthetic_samples[:N_SYNTHETIC_SAMPLES_TO_GENERATE]

print(f"Generation completed. Generated {len(synthetic_samples)} samples out of {total_attempts} attempts.")

if len(synthetic_samples) < N_SYNTHETIC_SAMPLES_TO_GENERATE:
    print(f"Warning: Generated fewer samples than requested. Consider lowering MODEL_CONFIDENCE_THRESHOLD.")

# Create final dataset
if synthetic_samples:
    df_synthetic = pd.DataFrame(synthetic_samples)
    df_model_driven = pd.concat([df_majority, df_synthetic]).sample(frac=1, random_state=RANDOM_STATE)
    
    print("\nNew model driven dataset class distribution:")
    print(df_model_driven['target'].value_counts(normalize=True))
    
    df_model_driven.to_csv(MODEL_DRIVEN_SYNTHETIC_PATH, index=False)
    print(f"Saved model driven synthetic dataset to {MODEL_DRIVEN_SYNTHETIC_PATH}")
else:
    print("No synthetic samples generated. Check your MODEL_CONFIDENCE_THRESHOLD and data.")

Starting batch generation...
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Error in batch processing: 'H2OFrame' object has no attribute 'remove'


c:\Project\Synthetic Intelligence\.venv\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Error in batch processing: 'H2OFrame' object has no attribute 'remove'


c:\Project\Synthetic Intelligence\.venv\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Error in batch processing: 'H2OFrame' object has no attribute 'remove'


...

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Error in batch processing: 'H2OFrame' object has no attribute 'remove'


c:\Project\Synthetic Intelligence\.venv\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Error in batch processing: 'H2OFrame' object has no attribute 'remove'
Generation completed. Generated 250000 samples out of 569926 attempts.


c:\Project\Synthetic Intelligence\.venv\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"



New model driven dataset class distribution:
target
1    0.848243
0    0.151757
Name: proportion, dtype: float64
Saved model driven synthetic dataset to ../data/02_processed/model_driven_synthetic.csv


In [9]:
h2o.cluster().shutdown()

H2O session _sid_8140 closed.
